In [ ]:
import requests
from datetime import datetime
import hashlib
import time
import sys, tweepy
from urllib.parse import quote

In [ ]:
consumer_key = 'DZf19sNuqpicIEOnqLtApleTX'
consumer_secret = 'XmHGBRQTL72yOU7szY5JaEuJXcdkOHFM7nBoQGJ6KHG5SuIRd7'
key = '1206805889488801793-XPDrK00fMkW1CmQsLpVwKmH5PT0qCI'
secret = 'TlO9jQc3vrvr1NnGCvqwZ8sJhTTp2jLkiV0URQNEUesUK'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(key, secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
cowin_base_url = 'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin'
telegram_api_url = 'https://api.telegram.org/bot1794047380:AAGrpZN8ITFj_wU5DYgqpyhSkmmkHtRLlig/sendMessage?chat_id=-1001311953767&text='
pin_codes = ['400068', '400092', '400103']
match_list = ['#mybmcvaccinationupdate', 'covishield', 'covaxin']
user_ids = ['1140642593002311680', '1140641768288899072']

In [ ]:
def cowin_url_response(pincode):
    cowin_url = cowin_base_url+'?pincode='+pincode+'&date='+datetime.now().strftime('%d-%m-%Y')
    response = requests.get(cowin_url)
    response_text = response.text.encode('utf 8')   #For hashing
    response_json = response.json()   #For getting individual center deets
    
    hashed_response = hashlib.sha256(response_text).hexdigest()
            
    return (hashed_response, response_json)

def send_telegram_message(message):
    telegram_url = telegram_api_url + message
    try: 
        response = requests.get(telegram_url)
    except:
        print('Telegram issue')    #added since Telegram would sometimes raise an error
    print(response.text)
    
def center_loop(response_json):
    for center in response_json['centers']:    #loop through each center
        center_name = center['name']
        pin = center['pincode']
        cat = center['fee_type']
        for session in center['sessions']:    #loop through each session in for a particular center
            min_age_limit = session['min_age_limit']
            available_1 = session['available_capacity_dose1'] 
            available_2 = session['available_capacity_dose2'] 
            vaccine_type = session['vaccine']
            vax_date = session['date']
            vax_date = datetime.strptime(vax_date,'%d-%m-%Y').strftime('%b %d, %Y')
            if available_1 > 0 or available_2 > 0:    #only if doses are available
                if min_age_limit == 18 and cat == 'Free' and vaccine_type == 'COVISHIELD':
                    print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 18+ at {center_name} in zip code {pin}')
                    send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 18-plus at {center_name} in zip code {pin}')
#                 elif min_age_limit == 30:
#                     print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 30+ at {center_name} in zip code {pin}')
#                     send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 30-plus at {center_name} in zip code {pin}')
#                 elif min_age_limit == 45:
#                     print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 45+ at {center_name} in zip code {pin}')
#                     send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 45-plus at {center_name} in zip code {pin}')
                else:
                    continue
              
def twitter_update(since_ids): 
    results = []
    for user, since_id in zip(user_ids, since_ids):
        results.append(api.user_timeline(user_id=user, since_id=since_id, tweet_mode='extended', exclude_replies=True, count=50))

    new_since_ids = since_ids

    for i, result in enumerate(results):    #loop for all accounts
        for j in range(len(result)):    #loop till all returned tweets are parsed for specific account
            if any(match in result[j].full_text.lower() for match in match_list) and result[j].lang=='en': 
                encode_text = quote(result[j].full_text, safe='')
                send_telegram_message('Tweet update from Ward R-N:\n\n' + encode_text ) if i==0 else send_telegram_message('Tweet update from Ward R-C:\n\n' + encode_text)
                new_since_ids[i] = result[j].id_str
    return(new_since_ids)

In [ ]:
old_state = []    #list of previous(initial) state(json response of requests.get for each pin code) for all 3 pincodes

#Initial availability list
for pin in pin_codes:    #perform task for each pincode provided in pin_codes list
    old_state.append(cowin_url_response(pin))
        
for state in old_state:
    center_loop(state[1])

initial_since_ids = ['1411249819125829637', '1410921234158608386']

# f = open('since_ids.txt', 'r')
# ids = f.read()
# initial_since_ids = id.rstrip(' ').split(' ')

new_since_ids = twitter_update(initial_since_ids)

# f = open('since_ids.txt','w')
# for ids in new_since_ids:
#     f.write(ids + ' ')

#Consecutive availability list after every 10secs
while True:
    time.sleep(10)
#   (t_i=3*n_r | t_i: minimum time interval b/w requests | n_r: no. of requests after every t_i)
#   since limit is 100 requests every 5mins(300secs)

    new_state = []    #list of new state after 10s (json response of requests.get for each pin code) for all 3 pincodes
    for pin in pin_codes:
        new_state.append(cowin_url_response(pin))
    
    for i in range(len(pin_codes)):
        if old_state[i][0] != new_state[i][0]:    #compare the hash of responses for old and new state for each individual pincode
            center_loop(new_state[i][1])
    
    old_state = new_state
    
    new_since_ids = twitter_update(new_since_ids)
    
#     new_since_ids = twitter_update(f.read().rstrip(' ').split(' '))    
#     for ids in new_since_ids:
#         f.write(ids + ' ')
   
    continue                
    

In [ ]:
#url for api call to cowin website - pincode wise
# https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode=400001&date=21-06-2021

In [ ]:
# url for test group on telegram with chat id @cowin68
# telegram_api_url = 'https://api.telegram.org/bot1881845500:AAHmKDdldowa0Ns7v2wdxhQ4HqBa5hr82tc/sendMessage?chat_id=@cowin68&text='